# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils
from neural_data_analysis.neural_analysis_tools.glm_tools import glm_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

## get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"

In [ ]:
reload(pn_helper_class)

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = pn_aligned_by_seg.PlanningAndNeuralSegmentAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
    pn.planning_data_by_point)
pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

# get planning_data by segment

In [ ]:
pn.prepare_seg_aligned_data(segment_duration=2, rebinned_max_x_lag_number=2)

In [ ]:
pn.rebinned_x_var

In [ ]:
pn.rebinned_y_var

In [ ]:
# for regression later
use_raw_spike_data_instead = False

pn.get_concat_data_for_regression(use_raw_spike_data_instead=True,
                                    use_lagged_raw_spike_data=False,
                                    apply_pca_on_raw_spike_data=False,
                                    num_pca_components=7)
pn.print_data_dimensions()

In [ ]:
stop!

In [ ]:
pn.rebinned_y_var.columns.tolist()

# GLM

In [ ]:
from neural_data_analysis.neural_analysis_tools.glm_tools.tpg import glm_bases, plot_glm_fit

In [ ]:
for key, item in data.items():
    try:
        print(key, item.shape)
    except:
        print(key, item)

In [ ]:
data = glm_on_multiff.simulate_multiff_trials(n_trials=8, trial_len=350, dt=0.01, seed=3)
res, design_df, metrics, meta = glm_on_multiff.fit_multiff_glm(
    dt=data["dt"], trial_ids=data["trial_ids"],
    cur_vis=data["cur_vis"], nxt_vis=data["nxt_vis"],
    cur_dist=data["cur_dist"], nxt_dist=data["nxt_dist"],
    cur_angle=data["cur_angle"], nxt_angle=data["nxt_angle"],
    heading=data["heading"], speed=data["speed"], curvature=data["curvature"],
    spike_counts=data["spike_counts"], l2=0.0, use_trial_FE=False, cluster_se=True,
)
print("Metrics:\n", {k: (v if k != 'per_trial_deviance' else '... DataFrame ...') for k, v in metrics.items()})
# plot_glm_fit.plot_fitted_kernels(res, design_df, meta, data["dt"])
# plot_glm_fit.plot_angle_tuning(res, design_df, meta, data["dt"])
plot_glm_fit.plot_angle_kernels_with_ci(res, design_df, meta, data["dt"], base_prefix='cur_angle')


# PGAM functions

## individual steps

### categorize variables

In [ ]:
pgam_inst = pgam_class.PGAMclass(pn.x_var, pn.y_var_reduced, pn.bin_width, pn.processed_neural_data_folder_path)

In [ ]:
pgam_inst.prepare_for_pgam(num_total_trials=10)

### temporal kernel

modified from PGAM_Tutorial.ipynb

In [ ]:
pgam_inst._add_temporal_features_to_model()

In [ ]:
help(gdh.smooths_handler.add_smooth)

### spatial variable

In [ ]:
pgam_inst._add_spatial_features_to_model()

### run

In [ ]:
pgam_inst.run_pgam(neural_cluster_number=10)

### post-processing

In [ ]:
pgam_inst.post_processing()

### save results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import BSpline

# Degree of spline
k = 3

# Example 1: Not enough start knots
knots1 = [0, 2, 4, 6, 8, 10, 10, 10, 10]
coeffs = np.ones(len(knots1) - k - 1)
spline1 = BSpline(knots1, coeffs, k)

# Example 2: Proper repeated start knots
knots2 = [0, 0, 0, 0, 2, 4, 6, 8, 10, 10, 10, 10]
coeffs2 = np.ones(len(knots2) - k - 1)
spline2 = BSpline(knots2, coeffs2, k)

x = np.linspace(-1, 11, 400)

plt.plot(x, spline1(x), label="Start knot once")
plt.plot(x, spline2(x), label="Start knot repeated")
plt.axvline(0, color='gray', linestyle='--', label="x = 0")
plt.legend()
plt.title("Effect of Repeating Start Knot")
plt.xlabel("x")
plt.ylabel("Spline Value")
plt.grid(True)
plt.show()


In [ ]:
pgam_inst.save_results()

In [ ]:
stop!

## iterate through all neurons

In [ ]:
pgam_inst = pgam_class.PGAMclass(pn.x_var, pn.y_var, pn.bin_width, pn.processed_neural_data_folder_path)

In [ ]:
for i in range(pn.x_var.shape[1]):
    print(f'neural_cluster_number: {i} out of {pn.x_var.shape[1]}')
    pgam_inst.streamline_pgam(neural_cluster_number=i, num_total_trials=10)